In [1]:
import os
from openai import OpenAI
import httpx

import inspect
from pprint import pprint
from diskcache import Cache
from pydantic import BaseModel, Field

from typing import Any
from typing import Literal
import jupyter_black
import json
import requests
import requests
from markdownify import markdownify as md

jupyter_black.load()

DEEPSEEK_API_KEY = os.environ["DEEPSEEK_API_KEY"]
DEEPSEEK_BASE_URL = "https://api.deepseek.com"
MODEL = "deepseek-chat"
cache = Cache("./cache")
client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_BASE_URL)

In [2]:
def to_openai_tool(func: callable) -> dict[str, Any]:
    sig = inspect.signature(func)
    if sig.parameters:
        param = next(iter(sig.parameters.values()))
        param_type = param.annotation
        schema = param_type.model_json_schema()
    else:
        param_type = None
        schema = {"type": "object", "properties": {}, "required": []}

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (param_type.__doc__ if param_type else func.__doc__ or ""),
            "parameters": schema,
        },
    }


@cache.memoize()
def llm_call(
    messages: list[dict[str, str]], tools: list[dict[str, Any]], model: str
) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools if tools else None,
        temperature=0.0,
        tool_choice="auto",
    )
    return response.choices[0].message

In [3]:
# get current weather
class GetWeather(BaseModel):
    latitude: float
    longitude: float
    temperature_unit: Literal["celsius", "fahrenheit"]


def get_current_weather(data: GetWeather) -> str:
    """Get the current weather in a given location."""
    resp = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": data.latitude,
            "longitude": data.longitude,
            "temperature_unit": data.temperature_unit,
            "current_weather": True,
        },
    )
    return json.dumps(resp.json())


# get current location
def get_current_location() -> str:
    return json.dumps(requests.get("http://ip-api.com/json?fields=lat,lon").json())


# calculate
class Calculate(BaseModel):
    formula: str = Field(
        description="Numerical expression to compute the result of, in Python syntax."
    )


def calculate(
    data: Calculate,
) -> str:
    """Calculate the result of a given formula."""
    return str(eval(data.formula))


TOOLS = [
    to_openai_tool(get_current_weather),
    to_openai_tool(get_current_location),
    to_openai_tool(calculate),
]
FUNC_TYPES = {
    get_current_weather: GetWeather,
    get_current_location: None,
    calculate: Calculate,
}

In [4]:
class ReACTAgent:
    def __init__(
        self,
        system: str = "",
        model: str = "deepseek-chat",
        tools: list[dict[str, Any]] | None = None,
        max_iters: int = 10,
    ):
        self.model = model
        self.tools = tools or []
        self.messages: list[dict[str, str]] = []
        if system:
            self.messages.append({"role": "system", "content": system})

        self.max_iters = max_iters

    def __call__(self, content: str) -> str:
        self.messages.append({"role": "user", "content": content})

        iterations = 0
        while True:
            iterations += 1
            message = self._send()
            self.messages.append(message)

            tool_calls = getattr(message, "tool_calls", None)
            if not tool_calls:
                return message.content

            for call in tool_calls:
                func = globals()[call.function.name]
                args_model = FUNC_TYPES[func]
                args = (
                    args_model.model_validate_json(call.function.arguments)
                    if call.function.arguments != "{}"
                    else None
                )
                print(f"** CALLING FUNCTION {func.__name__} **")
                print(f"*** ARGS ***\n{args}\n")

                result = func(args) if args else func()
                print(f"*** RESULT ***\n{result}\n***")

                self.messages.append(
                    {
                        "role": "tool",
                        "tool_call_id": call.id,
                        "content": result,
                    }
                )

            if iterations >= self.max_iters:
                raise Exception("Max iterations reached")

    def _send(self):
        return llm_call(self.messages, self.tools, self.model)

In [5]:
system_prompt = """
You are a helpful assistant who can answer multistep questions by sequentially calling functions. 

Follow a pattern of:
- THOUGHT (reason step-by-step about which function to call next)
- ACTION (call a function as a next step towards the final answer) 
- OBSERVATION (output of the function)

Reason step by step which actions to take to get to the answer. 

Only call functions with arguments coming verbatim from the user or the output of other functions.
"""

# question = "What's the current weather for my location? Give me the temperature in degrees Celsius and the wind speed in knots."
# bot = ReACTAgent(system=system_prompt, tools=TOOLS)
# response = bot(question)
# print("Response:", response)

In [6]:
class SearchWeb(BaseModel):
    """Search the web for a given query."""

    query: str
    max_results: int = 10


def search_web(data: SearchWeb) -> str:
    url = "https://api.marginalia.nu/{key}/search/{query}"
    rsp = httpx.get(url.format(key="public", query=data.query)).json()

    return str(rsp["results"][: data.max_results])


class ReadWebsite(BaseModel):
    """Read the content of a website and return it as text. (useful for further research)"""

    url: str


def read_website(data: ReadWebsite) -> str:

    html_content = requests.get(data.url).text
    return md(html_content)


TOOLS = [
    to_openai_tool(search_web),
    to_openai_tool(read_website),
]
FUNC_TYPES = {
    search_web: SearchWeb,
    read_website: ReadWebsite,
}

In [7]:
question = "What is the population of the city of Ancona?"

bot = ReACTAgent(
    system=system_prompt, tools=TOOLS
)  # model="deepseek-reasoner") Add support for this.
response = bot(question)
print("Response:", response)

** CALLING FUNCTION search_web **
*** ARGS ***
query='population of Ancona Italy 2023' max_results=10

*** RESULT ***
[{'url': 'https://savvysciencepublisher.com/jms/index.php/jppr/article/view/1031', 'title': 'Which Is The Role of Social Media in Web-Based Adolescent NonSuicidal Self-Injury (NSSI)?\n\t\t\t\t\t\t\t| Journal of Psychology and ...', 'description': 'Adolescent, Adolescence, Deliberate Self-Harm (DSH Non-suicidal Self-injuries (NSSI Self-harm, Social Media, Web In recent years, there has been a significant surge in non-suicidal self-injury (NSSI)-related content on the Internet. In fact, the technolog', 'quality': 6.551828672946033, 'format': 'html', 'details': [[]]}, {'url': 'https://en.wikipedia.org/wiki/List_of_mayors_of_Ancona', 'title': 'List of mayors of Ancona', 'description': "The Mayor of Ancona is an elected politician who, along with the Ancona's City Council, is accountable for the strategic government of Ancona in Marche, Italy. The current Mayor is Daniele Si